## Задание

Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу перефраза

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from datasets import load_dataset

In [4]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Using custom data configuration merionum--ru_paraphraser-4ab1b249bc6d5bf7
Reusing dataset json (/root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4ab1b249bc6d5bf7/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [6]:
print(corpus['train']['text_1'][0])

Цены на нефть восстанавливаются


In [7]:
print(corpus['train']['text_2'][0])

Парламент Словакии поблагодарил народы бывшего СССР за Победу


In [8]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [9]:
result = []

def compare(n):
    
    text1 = corpus['train']['text_1'][n]
    text2 = corpus['train']['text_2'][n]
    
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).numpy()
    
    p = proba[0]
    if p[1] > 0.5:
        x = "ДА"
    else:
        x = "НЕТ"
    
    result.append([text1, text2, p[1], x])
    df = pd.DataFrame(result, columns=['текст1', 'текст2', 'вероятность перефраза', 'перефраз'])
    
    return df

In [10]:
compare(0)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ


In [11]:
compare(1)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА


In [12]:
compare(7)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА


In [13]:
compare(10)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА


In [14]:
compare(15)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ


In [15]:
compare(20)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ
5,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144,ДА


In [16]:
compare(77)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ
5,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144,ДА
6,Вице-президент Гватемалы ушла в отставку из-за...,Вице-президент Гватемалы оставит свой пост из-...,0.972321,ДА


In [17]:
compare(100)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ
5,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144,ДА
6,Вице-президент Гватемалы ушла в отставку из-за...,Вице-президент Гватемалы оставит свой пост из-...,0.972321,ДА
7,Путин впервые объявил минуту молчания на парад...,Путин на параде Победы указал на опасности «бл...,0.053989,НЕТ


In [18]:
compare(141)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ
5,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144,ДА
6,Вице-президент Гватемалы ушла в отставку из-за...,Вице-президент Гватемалы оставит свой пост из-...,0.972321,ДА
7,Путин впервые объявил минуту молчания на парад...,Путин на параде Победы указал на опасности «бл...,0.053989,НЕТ
8,Пургин: Порошенко стоило приехать на парад Поб...,Парад Победы в Донецке посетили 40 тысяч человек,0.053516,НЕТ


In [19]:
compare(250)

,текст1,текст2,вероятность перефраза,перефраз
0,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,0.054301,НЕТ
1,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,0.841314,ДА
2,Васильева признана виновной в мошенничестве и ...,Васильева признана виновной в хищениях и отмыв...,0.981796,ДА
3,Пушков: у Обамы не хватило духа лично поздрави...,Пушков: Обама не нашел в себе духа лично поздр...,0.972294,ДА
4,Суд оправдал Васильеву в хищении акций на 2 мл...,Суд оправдал Васильеву в мошенничестве со здан...,0.196370,НЕТ
5,Лидер либдемов Ник Клегг подал в отставку посл...,Лидер британских лейбористов подал в отставку ...,0.919144,ДА
6,Вице-президент Гватемалы ушла в отставку из-за...,Вице-президент Гватемалы оставит свой пост из-...,0.972321,ДА
7,Путин впервые объявил минуту молчания на парад...,Путин на параде Победы указал на опасности «бл...,0.053989,НЕТ
8,Пургин: Порошенко стоило приехать на парад Поб...,Парад Победы в Донецке посетили 40 тысяч человек,0.053516,НЕТ
9,УФСИН: массовый бунт заключенных в башкирской ...,УФСИН сообщила о подавлении бунта в башкирской...,0.954364,ДА
